In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.cm as cm
from datetime import timedelta
import os

CLEAN_PATH = "/workspace/data/00/StatsAllGroup_cleaned.parquet"
df = pd.read_parquet(CLEAN_PATH)

# ★ 01 は FP 専用
df = df[df["position_group"] == "FP"].copy()

print(f"✅ Loaded FP data: {df.shape[0]} rows, {df['athlete_name'].nunique()} athletes")



✅ Loaded FP data: 65471 rows, 65 athletes


In [2]:
# ========================================
# 0. timeline（日付 × day_type）生成
# ========================================
all_dates = pd.date_range(df["date_"].min(), df["date_"].max(), freq='D')

daily_counts = df.groupby("date_")["athlete_id"].nunique()
daily_counts = daily_counts.reindex(all_dates, fill_value=0)

total_athletes = df["athlete_id"].nunique()
daily_percent = daily_counts / total_athletes * 100

OFF_TH = 5
MID_TH = 20

def label_day(p):
    if p < OFF_TH:
        return "Off"
    elif p < MID_TH:
        return "Mid"
    else:
        return "Season"

day_type = daily_percent.apply(label_day)

timeline = pd.DataFrame({
    "date_": all_dates,
    "day_type": day_type.values
})

print("📌 Generated timeline:", timeline["day_type"].value_counts())

df = df.merge(timeline, on="date_", how="left")
print("📌 Merged timeline into df")

📌 Generated timeline: day_type
Season    672
Off       313
Mid        13
Name: count, dtype: int64
📌 Merged timeline into df


In [3]:
# ========================================
# 1. Season 日のみ使用（Season-aware）
# ========================================
df_season = df[df["day_type"] == "Season"].copy()

player_summary = (
    df_season.groupby("athlete_id")
      .agg(
          athlete_name=("athlete_name", "first"),
          season_count=("date_", "count"),
          season_unique_days=("date_", "nunique"),
          first_season_date=("date_", "min"),
          last_season_date=("date_", "max"),
      )
      .reset_index()
)


player_summary["season_duration_days"] = (
    player_summary["last_season_date"] - player_summary["first_season_date"]
).dt.days + 1

player_summary["season_density_ratio"] = (
    player_summary["season_unique_days"] / player_summary["season_duration_days"]
)

print("📌 Season-based player summary created.")

📌 Season-based player summary created.


In [4]:
# ========================================
# 2. Season 版 3ヶ月45日ルールで valid_players を決定
# ========================================
window_days = 90
min_days_in_window = 45

valid_players = []

for athlete_id, sub in df_season.groupby("athlete_id"):
    dates = sub["date_"].drop_duplicates().sort_values()
    for start in dates:
        end = start + timedelta(days=window_days)
        if (dates.between(start, end).sum() >= min_days_in_window):
            valid_players.append(athlete_id)
            break


print(f"🎯 VALID PLAYERS（Season-aware 3ヶ月45日）: {len(valid_players)} 名")


🎯 VALID PLAYERS（Season-aware 3ヶ月45日）: 53 名


In [5]:
player_summary_valid = player_summary[
    player_summary["athlete_id"].isin(valid_players)
].copy()


In [6]:
# ---------------------------------------------------------
# Rule 1：Season出現日数
# ---------------------------------------------------------
player_summary_valid["season_days"] = player_summary_valid["season_unique_days"]

player_summary_valid["rule1_category"] = pd.cut(
    player_summary_valid["season_days"],
    bins=[0, 10, 30, 10000],
    labels=["exclude", "partial", "full"]
)

print("✔ Rule1分布：")
print(player_summary_valid["rule1_category"].value_counts())

✔ Rule1分布：
rule1_category
full       53
exclude     0
partial     0
Name: count, dtype: int64


In [7]:
# ---------------------------------------------------------
# Rule 2：平均距離（600 / 1200m閾値）
# ---------------------------------------------------------
mean_dist = (
    df_season.groupby("athlete_id")["total_distance"]
    .mean()
    .rename("mean_season_distance")
)

player_summary_valid = player_summary_valid.merge(
    mean_dist, on="athlete_id", how="left"
)


player_summary_valid["rule2_adjusted"] = player_summary_valid["rule1_category"]

player_summary_valid.loc[
    player_summary_valid["mean_season_distance"] < 600,
    "rule2_adjusted"
] = "exclude"

player_summary_valid.loc[
    (player_summary_valid["mean_season_distance"] >= 600) &
    (player_summary_valid["mean_season_distance"] < 1200),
    "rule2_adjusted"
] = "partial"


print("\n✔ Rule2分布：")
print(player_summary_valid["rule2_adjusted"].value_counts())


✔ Rule2分布：
rule2_adjusted
full       30
partial    23
exclude     0
Name: count, dtype: int64


In [8]:
# ---------------------------------------------------------
# Rule 3：クラスタリング（cluster_center で自動判定）
# ---------------------------------------------------------
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
import numpy as np

cluster_features = player_summary_valid[[
    "season_days",
    "mean_season_distance",
    "season_density_ratio"
]].fillna(0)

scaler = StandardScaler()
X = scaler.fit_transform(cluster_features)

kmeans = KMeans(n_clusters=3, random_state=42)
player_summary_valid["cluster"] = kmeans.fit_predict(X)

centers = kmeans.cluster_centers_
strength = centers.sum(axis=1)
ranked_clusters = np.argsort(strength)

cluster_to_label = {
    ranked_clusters[0]: "exclude",
    ranked_clusters[1]: "partial",
    ranked_clusters[2]: "full"
}

player_summary_valid["cluster_label"] = player_summary_valid["cluster"].map(cluster_to_label)

print("\n✔ Rule3 クラスタ割当：", cluster_to_label)
print(player_summary_valid["cluster_label"].value_counts())


✔ Rule3 クラスタ割当： {1: 'exclude', 0: 'partial', 2: 'full'}
cluster_label
partial    29
exclude    22
full        2
Name: count, dtype: int64


/opt/conda/lib/python3.11/site-packages/sklearn/cluster/_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)


In [9]:
# ---------------------------------------------------------
# Final 統合カテゴリ
# ---------------------------------------------------------
priority = ["exclude", "partial", "full"]

def combine_labels(row):
    candidates = [row["rule2_adjusted"], row["cluster_label"]]
    return max(candidates, key=lambda x: priority.index(x))

player_summary_valid["final_category"] = player_summary_valid.apply(combine_labels, axis=1)

print("\n==============================")
print("🎉 Final 選手分類（valid_playersのみ）")
print("==============================")
print(player_summary_valid["final_category"].value_counts())



🎉 Final 選手分類（valid_playersのみ）
final_category
full       30
partial    23
Name: count, dtype: int64


In [10]:
# ---------------------------------------------------------
# Final sanity check
# ---------------------------------------------------------
assert player_summary_valid["athlete_id"].is_unique
print("✅ athlete_id uniqueness OK")


✅ athlete_id uniqueness OK


In [11]:
# ========================================
# 3. 保存（01の出力はこの1つだけ）
# ========================================
PLAYER_DIR = "/workspace/data/01"
os.makedirs(PLAYER_DIR, exist_ok=True)

csv_path = f"{PLAYER_DIR}/fp_player_final_categories.csv"
parquet_path = f"{PLAYER_DIR}/fp_player_final_categories.parquet"

player_summary_valid.to_csv(csv_path, index=False, encoding="utf-8-sig")
player_summary_valid.to_parquet(parquet_path, index=False)

print("\n📁 Saved:")
print(" -", csv_path)
print(" -", parquet_path)
print("✨ Notebook 01 完全完了（CSV + Parquet 保存）")


📁 Saved:
 - /workspace/data/01/fp_player_final_categories.csv
 - /workspace/data/01/fp_player_final_categories.parquet
✨ Notebook 01 完全完了（CSV + Parquet 保存）
